In [1]:
#r "nuget: Microsoft.ML"
#r "nuget: Microsoft.ML.LightGbm"
#load "./Modules/MLWrapper.fs"
open System.IO
open System.Net
open Microsoft.ML
open Microsoft.ML.Data
open Microsoft.ML.Transforms
open FunctionalMl

Installed package Microsoft.ML version *

Installed package Microsoft.ML.LightGbm version *

In [2]:
if not <| File.Exists("arrhythmia.data") then
    use client = new WebClient()
    client.DownloadFile("https://archive.ics.uci.edu/ml/machine-learning-databases/arrhythmia/arrhythmia.data", "arrhythmia.data")

printfn "Data file has %d lines" <| File.ReadLines("arrhythmia.data").Count()
File.ReadLines("arrhythmia.data")
|> Seq.take 5

Data file has 452 lines


index,value
0,"75,0,190,80,91,193,371,174,121,-16,13,64,-2,?,63,0,52,44,0,0,32,0,0,0,0,0,0,0,44,20,36,0,28,0,0,0,0,0,0,52,40,0,0,0,60,0,0,0,0,0,0,52,0,0,0,0,0,0,0,0,0,0,0,0,56,36,0,0,32,0,0,0,0,0,0,48,32,0,0,0,56,0,0,0,0,0,0,80,0,0,0,0,0,0,0,0,0,0,0,0,40,52,0,0,28,0,0,0,0,0,0,0,48,48,0,0,32,0,0,0,0,0,0,0,52,52,0,0,36,0,0,0,0,0,0,0,52,48,0,0,32,0,0,0,0,0,0,0,56,44,0,0,32,0,0,0,0,0,0,-0.2,0.0,6.1,-1.0,0.0,0.0,0.6,2.1,13.6,30.8,0.0,0.0,1.7,-1.0,0.6,0.0,1.3,1.5,3.7,14.5,0.1,-5.2,1.4,0.0,0.0,0.0,0.8,-0.6,-10.7,-15.6,0.4,-3.9,0.0,0.0,0.0,0.0,-0.8,-1.7,-10.1,-22.0,0.0,0.0,5.7,-1.0,0.0,0.0,-0.1,1.2,14.1,22.5,0.0,-2.5,0.8,0.0,0.0,0.0,1.0,0.4,-4.8,-2.7,0.1,-6.0,0.0,0.0,0.0,0.0,-0.8,-0.6,-24.0,-29.7,0.0,0.0,2.0,-6.4,0.0,0.0,0.2,2.9,-12.6,15.2,-0.1,0.0,8.4,-10.0,0.0,0.0,0.6,5.9,-3.9,52.7,-0.3,0.0,15.2,-8.4,0.0,0.0,0.9,5.1,17.7,70.7,-0.4,0.0,13.5,-4.0,0.0,0.0,0.9,3.9,25.5,62.9,-0.3,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,8"
1,"56,1,165,64,81,174,401,149,39,25,37,-17,31,?,53,0,48,0,0,0,24,0,0,0,0,0,0,0,64,0,0,0,24,0,0,0,0,0,0,32,24,0,0,0,40,0,0,0,0,0,0,48,0,0,0,0,0,0,0,0,0,0,0,0,44,20,0,0,24,0,0,0,0,0,0,0,60,0,0,0,20,0,0,0,0,0,0,0,24,52,0,0,16,0,0,0,0,0,0,0,32,52,0,0,20,0,0,0,0,0,0,0,44,48,0,0,32,0,0,0,0,0,0,0,48,44,0,0,32,0,0,0,0,0,0,0,48,40,0,0,28,0,0,0,0,0,0,0,48,0,0,0,28,0,0,0,0,0,0,-0.6,0.0,7.2,0.0,0.0,0.0,0.4,1.5,17.2,26.5,0.0,0.0,5.5,0.0,0.0,0.0,0.1,1.7,17.6,29.5,0.3,-1.6,0.9,0.0,0.0,0.0,-0.3,0.4,-1.5,1.3,0.1,-6.4,0.0,0.0,0.0,0.0,-0.3,-1.6,-15.3,-25.5,-0.3,0.0,4.2,-0.9,0.0,0.0,0.4,0.7,8.3,12.3,0.2,0.0,2.2,0.0,0.0,0.0,-0.2,0.8,6.6,11.7,0.4,0.0,1.0,-8.8,0.0,0.0,0.5,-0.6,-21.6,-26.8,0.4,0.0,2.6,-7.9,0.0,0.0,0.8,2.0,-16.4,1.2,0.0,0.0,5.8,-7.7,0.0,0.0,0.9,3.8,-5.7,27.7,-0.2,0.0,9.5,-5.0,0.0,0.0,0.5,2.6,11.8,34.6,-0.4,0.0,11.0,-2.4,0.0,0.0,0.4,2.6,21.6,43.4,-0.5,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,6"
2,"54,0,172,95,138,163,386,185,102,96,34,70,66,23,75,0,40,80,0,0,24,0,0,0,0,0,0,20,56,52,0,0,40,0,0,0,0,0,0,28,116,0,0,0,52,0,0,0,0,0,0,52,64,0,0,0,88,0,0,0,0,0,0,0,36,92,0,0,24,0,0,0,0,0,0,0,128,0,0,0,24,0,1,0,0,0,0,0,24,36,76,0,100,0,0,0,0,0,0,0,40,28,60,0,96,0,0,0,0,0,0,0,48,20,56,24,32,0,0,0,0,0,0,0,44,88,0,0,28,0,0,0,0,0,0,0,44,76,0,0,28,0,0,0,0,0,0,0,44,72,0,0,24,0,0,0,0,0,0,1.0,0.0,4.5,-2.8,0.0,0.0,0.3,2.5,-2.2,19.8,0.8,-0.4,6.4,-1.3,0.0,0.0,0.7,2.7,14.2,37.9,-0.2,-0.6,4.4,0.0,0.0,0.0,0.5,0.2,24.7,26.2,-1.0,-5.3,1.8,0.0,0.0,0.0,-0.5,-2.5,-8.0,-28.5,0.5,0.0,1.7,-2.7,0.0,0.0,-0.2,1.0,-9.4,-1.2,0.4,0.0,4.9,0.0,0.0,0.0,0.6,1.4,31.3,42.7,-0.8,0.0,0.7,-3.8,6.5,0.0,0.3,-3.3,18.7,-13.6,-0.9,0.0,2.2,-4.1,7.4,0.0,0.5,-2.4,20.9,-2.6,0.0,0.0,5.8,-4.1,4.0,-0.5,0.4,0.3,20.4,23.3,0.7,0.0,10.0,-5.7,0.0,0.0,0.5,2.2,-3.0,20.7,1.3,0.0,11.1,-3.4,0.0,0.0,0.4,3.4,11.5,48.2,0.9,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,10"
3,"55,0,175,94,100,202,380,179,143,28,11,-5,20,?,71,0,72,20,0,0,48,0,0,0,0,0,0,0,64,36,0,0,36,0,0,0,0,0,0,20,52,48,0,0,56,0,0,0,0,0,0,64,32,0,0,0,72,0,0,0,0,0,0,0,60,12,0,0,44,0,0,0,0,0,0,0,60,44,0,0,32,0,0,0,0,0,0,56,0,0,0,0,0,0,0,0,0,0,0,0,40,44,0,0,20,0,0,0,0,0,0,0,52,40,0,0,32,0,0,0,0,0,0,0,56,48,0,0,36,0,0,0,0,0,0,0,60,48,0,0,36,0,0,0,0,0,0,0,64,40,0,0,40,0,0,0,0,0,0,0.9,0.0,7.8,-0.7,0.0,0.0,1.1,1.9,27.3,45.1,0.1,0.0,9.1,-2.6,0.0,0.0,0.4,1.5,24.5,36.8,-0.4,-0.4,1.6,-2.2,0.0,0.0,-1.0,-0.9,-1.5,-9.2,-0.4,-8.2,1.8,0.0,0.0,0.0,-0.7,-1.7,-23.4,-35.6,0.9,0.0,3.2,-0.4,0.0,0.0,0.7,1.2,9.4,18.0,-0.1,0.0,5.1,-2.5,0.0,0.0,0.3,0.6,9.8,12.6,1.6,-6.5,0.0,0.0,0.0,0.0,-0.4,-0.4,-18.2,-22.4,2.1,0.0,1.2,-6.9,0.0,0.0,-0.5,2.9,-12.7,18.0,0.7,0.0,9.0,-7.9,0.0,0.0,0.1,4.1,7.6,51.0,0.4,0.0,15.0,-5.5,0.0,0.0,0.1,3.3,28.8,63.1,0.1,0.0,15.2,-3.7,0.0,0.0,0.6,3.0,36.8,68.0,0.1,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1"
4,"75,0,190,80,88,181,360,177,103,-16,13,61,3,?,?,0,48,40,0,0,28,0,0,0,0,0,0,0,40,24,0,0,24,0,0,0,0,0,0,52,36,0,0,0,60,0,0,0,0,0,0,48,28,0,0,0,56,0,0,0,0,0,0,0,48,36,0,0,28,0,0,0,0,0,0,44,0,0,0,0,0,0,0,0,0,0,0,88,0,0,0,0,0,0,0,0,0,0,0,0,40,52,0,0,28,0,0,0,0,0,0,0,48,48,0,0,32,0,0,0,0,0,0,0,48,52,0,0,32,0,0,0,0,0,0,0,52,44,0,0,28,0,0,0,0,0,0,0,52,48,0

In [3]:
let singleColumn index =
    new TextLoader.Column(string index, DataKind.Single, index)

let columns =
    seq { 0..278 }
    |> Seq.map singleColumn
    |> Seq.append [ new TextLoader.Column("Label", DataKind.Int32, 279) ]
    |> Seq.toArray

In [4]:
let featureColumns =
    seq { 0..278 }
    |> Seq.map string
    |> Seq.toArray

In [5]:
let textLoader = ML.context.Data.CreateTextLoader(columns, hasHeader = false, separatorChar = ',')

In [6]:
let allData = textLoader.Load("arrhythmia.data")

In [7]:
let trainData, testData =
    ML.shuffle allData
    |> ML.split 0.2

In [8]:
trainData.GetColumn<int>("Label")
|> Seq.distinct

index,value
0,1
1,14
2,10
3,2
4,4
5,5
6,16
7,6
8,3
9,9


In [9]:
trainData.GetColumn<int>("Label")
|> Seq.countBy id
|> Seq.map (fun (x, y) -> {| Label=x; Count=y |})
|> Seq.sortBy (fun x -> x.Label)

index,Count,Label
0,190,1
1,39,2
2,13,3
3,15,4
4,11,5
5,20,6
6,3,7
7,2,8
8,8,9
9,38,10


In [10]:
let pipeline =
    EstimatorChain()
    |> ML.append <| ML.mapValueToKey "Label" "Label"
    |> ML.append <| ML.concatenate "Features" featureColumns
    |> ML.append <| ML.replaceMissingWithMean "Features" "Features"
    |> ML.append <| ML.normalizeMinMax "Features" "FeaturesNorm"

In [11]:
let transformer =
    pipeline
    |> ML.fit trainData // Fit our pipeline on the training data

In [12]:
let transformedData =
    trainData
    |> ML.transform transformer

In [13]:
trainData.Preview().RowView
|> Seq.take 5

index,Values
0,"[ Label: 1, 0: 48, 1: 0, 2: 162, 3: 84, 4: 80, 5: 154, 6: 354, 7: 171, 8: 85 ... (271 more) ]"
1,"[ Label: 1, 0: 34, 1: 1, 2: 155, 3: 54, 4: 86, 5: 127, 6: 369, 7: 157, 8: 78 ... (271 more) ]"
2,"[ Label: 1, 0: 46, 1: 1, 2: 158, 3: 58, 4: 70, 5: 120, 6: 353, 7: 122, 8: 52 ... (271 more) ]"
3,"[ Label: 14, 0: 22, 1: 1, 2: 165, 3: 52, 4: 108, 5: 184, 6: 406, 7: 218, 8: 114 ... (271 more) ]"
4,"[ Label: 10, 0: 18, 1: 0, 2: 175, 3: 60, 4: 102, 5: 135, 6: 379, 7: 167, 8: 73 ... (271 more) ]"


In [14]:
transformedData.GetColumn<VBuffer<single>>("FeaturesNorm")
|> Seq.take 5
|> Seq.map (fun vec -> vec.DenseValues())

index,
0,"[ 0.57831323, 0, 0.8526316, 0.47727275, 0.4255319, 0.48125002, 0.69548136, 0.4488189, 0.4166667, 0.22674419 ... (269 more) ]"
1,"[ 0.26506025, 1, 0.8684211, 0.29545456, 0.5744681, 0.575, 0.79764247, 0.5721785, 0.5588235, 0.13953489 ... (273 more) ]"
2,"[ 0.26506025, 1, 0.8684211, 0.29545456, 0.5744681, 0.575, 0.79764247, 0.5721785, 0.5588235, 0.13953489 ... (269 more) ]"
3,"[ 0.26506025, 1, 0.8684211, 0.29545456, 0.5744681, 0.575, 0.79764247, 0.5721785, 0.5588235, 0.13953489 ... (269 more) ]"
4,"[ 0.21686746, 0, 0.9210527, 0.3409091, 0.5425532, 0.421875, 0.74459726, 0.43832022, 0.35784316, 0.5290698 ... (269 more) ]"


In [15]:
let estimator = 
    ML.context.MulticlassClassification.Trainers.LightGbm(featureColumnName = "FeaturesNorm", learningRate = Nullable<float>(0.1))
    |> ML.downcastEstimator

In [16]:
let model =
    trainData // Begin with the training data
    |> ML.transform transformer // Transform using the transformer built above
    |> ML.crossValidateMulticlassClassification estimator 3 // 3-fold cross-validation
    |> ML.printMulticlassClassificationCvMetrics // Print cross-fold metrics
    |> Seq.maxBy (fun cvResult -> cvResult.Metrics.MicroAccuracy)
    |> fun cvResult -> cvResult.Model

------------------
Cross Validation Metrics
------------------
Accuracy: 0.429219
Log Loss: 1.223530


In [17]:
model
|> ML.transform <| ML.transform transformer testData // Transform the test data and get predictions
|> ML.context.MulticlassClassification.Evaluate // Get test metrics
|> ML.printMulticlassClassificationMetrics

------------------
Test Metrics
------------------
Accuracy: 0.648485
Log Loss: 0.795972
Confusion Matrix:

Confusion table
          ||========================================================================================================
PREDICTED ||     0 |     1 |     2 |     3 |     4 |     5 |     6 |     7 |     8 |     9 |    10 |    11 |    12 | Recall
TRUTH     ||========================================================================================================
        0 ||    47 |     0 |     2 |     1 |     0 |     2 |     3 |     0 |     0 |     0 |     0 |     0 |     0 | 0.8545
        1 ||     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 | 0.0000
        2 ||     1 |     0 |    10 |     0 |     1 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 | 0.8333
        3 ||     3 |     0 |     0 |     2 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 |     0 | 0.4000
        4 ||     0 |  

In [18]:
[<CLIMutable>]
type MulticlassClassificationPrediction = { LabelValue: int; Score : single[]; PredictedLabelValue : int }

let sampleData =
    testData
    |> ML.shuffle
    |> ML.transform transformer
    |> ML.transform model

let mapValues =
    EstimatorChain()
    |> ML.append <| ML.mapKeyToValue "PredictedLabel" "PredictedLabelValue"
    |> ML.append <| ML.mapKeyToValue "Label" "LabelValue"
    |> ML.fit sampleData

let samplePredictions = mapValues.Transform(sampleData)
ML.context.Data.CreateEnumerable<MulticlassClassificationPrediction>(samplePredictions, reuseRowObject = false)
|> Seq.take 5

index,LabelValue,Score,PredictedLabelValue
0,1,"[ 0.97704, 0.0011728159, 0.0008744632, 0.0014531577, 0.00060047733, 0.0019211222, 0.010387571, 0.0002906195, 0.001139807, 0.0016155671 ... (3 more) ]",1
1,10,"[ 0.061290126, 0.0018989801, 0.92797506, 0.00032050128, 0.00041107112, 0.0015196834, 0.0012822984, 0.00055546727, 0.0007636303, 0.0010276618 ... (3 more) ]",10
2,9,"[ 0.013076769, 0.00077391264, 0.0011445633, 0.27049413, 0.00057964656, 0.0003855137, 0.052401487, 0.0035830052, 0.028297214, 0.6226301 ... (3 more) ]",9
3,1,"[ 0.98464394, 0.0009561922, 0.00071388966, 0.000275566, 0.00088409294, 0.0039644614, 0.0005799404, 0.0011390178, 0.0008004672, 0.0011510475 ... (3 more) ]",1
4,1,"[ 0.9322044, 0.0016002799, 0.0016179279, 0.010951799, 0.0018904129, 0.0008404474, 0.0066934773, 0.03589578, 0.0015749462, 0.0023853008 ... (3 more) ]",1
